In [ ]:
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import re
import pandas as pd
import time

In [ ]:
page = 1
#data container, storing every rows/wordcounts for each sub pages
#to be changed to DataFrame
dict_title_additional_info = {'Job Title':[], 'Career Level':[],'Qualification':[],'Years of Experience':[], "Industry":[]}
dict_wordcount = {}
url = 'https://hk.jobsdb.com/hk/en/Search/FindJobs?JSRV=1&JobCat=311%2C134%2C141&page='
html = requests.get(url + str(page))
soup = bs(html.text, 'html.parser')
total_pg = int(re.sub(r'\D','', soup.find(class_='sQuda_6').get_text())) // 30 + 1 #divided by 30 since one page has 30 sublink


# while page < 5: (for code testing)
while page < total_pg:
    links = []
    url1 = url + str(page)
    html = requests.get(url1)
#     time.sleep(10) (optional)
    soup = bs(html.text, 'html.parser')
    x = soup.find_all(class_="FYwKg _1GAuD C6ZIU_6 _6ufcS_6 _27Shq_6 sQuda_6 _2WTa0_6")#scrape all subpages' link
    for i in x:
        links.append(i.a['href'])

    for i in links:
        #l is list of links in one page
        #i is every sub pages. 
        
        driver = webdriver.Chrome(executable_path='C:/Users/USER/OneDrive/Xccelerate/chromedriver.exe')
        driver.get(i)
        subhtml = driver.page_source
        soup_post = bs(subhtml, "html.parser")
        #post = requests.get(i)
        #soup_post = bs(post.text, 'html.parser')
        
        
        #Codes of What we scraped start here#
        
        #Job title
        z = soup_post.find_all(class_='FYwKg C6ZIU_6 _3nVJR_6 _642YY_6 _27Shq_6 _2k6I7_6')
        dict_title_additional_info['Job Title'].append(z)
        
        #Additional info
        y = soup_post.find_all(class_='FYwKg _32Ekc _2fqoM_6 _1hqiH_6')

        for i in y: #data cleaning
            dict_title_additional_info['Career Level'].append(y[0].text.replace('Career Level',''))
            dict_title_additional_info['Qualification'].append(y[1].text.replace('Qualification',''))
            dict_title_additional_info['Years of Experience'].append(y[2].text.replace('Years of Experience',''))
            dict_title_additional_info['Industry'].append(y[-1].text.replace('Industry',''))  
        
        
    print(dict_title_additional_info)
    page += 1

df_title_additional_info = pd.DataFrame.from_dict(dict_title_additional_info)
df_title_additional_info.to_csv('df_info_test.csv')